In [5]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from tqdm import tqdm

from model import VGG

In [2]:
from dataset import GrapeDataset

In [3]:
def create_annotations(dir_path):
    """ 
    Returns a Dataframe with images and labels
    """
    X = []
    y = []
    i = 0
    for path in tqdm(sorted(os.listdir(dir_path))):
        if not path.startswith('.'):
            for file in sorted(os.listdir(dir_path + path)):
                if not file.startswith('.'):
                    X.append(path+'/'+file)
                    y.append(i)
            i +=1
    print(f'{len(X)} images loaded from {dir_path} directory.')
    return pd.DataFrame({'filename':X,
                        'label':y})

In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
test_path = 'data/test/'
train_path = 'data/train/'    
# Hyperparameters
batch_size = 4
learning_rate = 1e-3
num_epochs = 10
# Loading data
ann_train = create_annotations(train_path)
ann_test = create_annotations(test_path)

transform = transforms.Compose([ transforms.ToTensor(), transforms.Resize((224,224))])
train_data = GrapeDataset(img_dir=train_path, ann_df=ann_train,transform=transform)
test_data = GrapeDataset(img_dir=test_path, ann_df=ann_test,transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Loading the model
model = VGG(num_classes=4)
model.to(device)
# Loss and Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

100%|██████████| 5/5 [00:00<00:00, 114.57it/s]


7222 images loaded from data/train/ directory.


100%|██████████| 5/5 [00:00<00:00, 401.84it/s]


1805 images loaded from data/test/ directory.


In [22]:
X, y = next(iter(train_loader))

In [23]:
device = "mps"
model.to(device)

VGG(
  (conv_stack): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(

In [24]:
num_epochs = 1000
for epoch in range(num_epochs):
    X, y = X.to(device), y.to(device)
    preds = model(X)
    loss = loss_fn(preds, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)

tensor(1.4733, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.2386, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(1.0031, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.9614, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.8681, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7757, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.7149, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6477, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6539, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.6078, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4467, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4094, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3855, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.4317, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3499, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.2183, device='mps:0', grad_fn=<NllLossBackward0>)
tensor(0.3192, device='mps:0', grad_fn=<NllLossBackward0

In [25]:
preds

tensor([[-2.3598,  6.5041, -1.5183, -2.7046],
        [-1.2176, -1.3093,  6.5731, -2.0980],
        [-1.3654,  6.7710, -0.9830, -2.8361],
        [ 6.5101, -2.6940, -2.0533, -2.6860]], device='mps:0',
       grad_fn=<LinearBackward0>)

In [31]:
y

tensor([1, 2, 1, 0], device='mps:0')

In [17]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    # Train
    model.train()
    for idx, (X,y) in loop:
        X, y = X.to(device), y.to(device)

        preds = model(X)
        loss = loss_fn(preds, y)
        train_running_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_description(f"Epoch:[{epoch}/{num_epochs}](Train)")
        loop.set_postfix(train_loss=loss.item())

    train_loss = train_running_loss / (idx + 1)

    # Val
    val_running_loss = 0.0
    run_accuracy = 0
    model.eval()
    loop = tqdm(enumerate(test_loader), total=len(test_loader), leave=False)
    with torch.no_grad():
        for idx, (X,y) in loop:
            X, y = X.to(device), y.to(device)

            preds = model(X)
            loss = loss_fn(preds, y)
            val_running_loss += loss.item()
            # accuracy = calc_accuracy(preds, mask, cfg.THR)
            # run_accuracy += accuracy.item()
            loop.set_description(f"Epoch:[{epoch}/{num_epochs}](Val)")
            # loop.set_postfix(val_loss=loss.item(), acc=accuracy.item())

KeyboardInterrupt: 